# Self Study 7: Solow and the Steady State

## Section 1: The Solow Model

$$
Y = A \cdot K^\alpha \cdot L^{1-\alpha} 
$$

$$
\frac{dK}{dt} = s \cdot Y - \delta \cdot K
$$

$$
\frac{dL}{dt} = n \cdot L
$$

### Task 1: Explain the meaning of each variable in the equations above.

Note that the variable names may be different from the slides

- $ K $ is the amount of Capital in the economy
- $ L $ is the amount of Labour force in the economy. i.e. how many people are working
- $ Y $ is the total production of the economy. (GDP)
- $ s $ is the savings rate. i.e. which percentage of the total production that is not consumed but saved in investments
- $ \delta $ is the depreciation rate. i.e. how much of the capital that loses it's value this time unit
- $ n $ is the labor growth factor. i.e. how much the labour force increases

### Task 2: Write down the intensive form of the model.

The above equations are in extensive form, which means they are in terms of *total* output, total capital, total labour. To simplify the analysis and reasoning about the implications of the model, we can convert the equations to intensive form, which means they are in terms of output *per capita*, capital per worker, and labour per worker.

It is common to use lower case letters to denote intensive variables, and upper case letters to denote extensive variables. For example, $k$ is capital per worker, and $K$ is total capital.

Write down the intensive form of the model:

$$
y = A \cdot \frac{K^\alpha \cdot L^{1-\alpha}}{L} = A \cdot \frac{K^{\alpha} \cdot L^{1-\alpha}}{L^{\alpha} \cdot L^{1-\alpha}} = A \cdot k^{\alpha}
$$

$$
\frac{dk}{dt} = \frac{sY-\delta K}{L} + K \cdot -L^{-2} \cdot nL = \frac{sY-\delta K - nK}{L} = sy- (\delta + n)k
$$

$$
\frac{dl}{dt} = \frac{d}{dt}(\frac{L}{L}) = \frac{d}{dt}1 = 0
$$

### Task 3: Plot both the extensive and intensive form of the model.

Plot Capital and Output per worker over time for the extensive and intensive form of the model.



In [10]:
import numpy as np

def output_per_worker(A, k, alpha):
    return A * k ** alpha

def total_output(A, K, alpha, L):
    return A * K**(alpha)*L**(1-alpha)

def intensive_dynamics(y,t,A, alpha, delta, n, s):
    output = output_per_worker(A, y[1], alpha)
    dk_dt = s * y[0] - n* y[1] - delta * y[1]
    return [output, dk_dt]

def extensive_dynamics(y,t,A,alpha,delta,n,s):
    output = total_output(A, y[1], alpha, y[2])
    dK_dt = s * y[0] - delta * y[1]
    dL_dt = n*y[2]
    return [output, dK_dt, dL_dt]

init_conditions = [100, 70, 50]


In [15]:
import matplotlib.pyplot as plt
from ipywidgets import interact
from scipy.integrate import odeint

t = np.linspace(0, 5, 50)

@interact(A=(0,10,0.1), alpha=(0,1,0.01), delta=(0,1,0.01), n=(0,4,0.1), s=(0,1,0.01))
def plot_wage_dynamics(A=1, alpha=0.4, delta=0.2, n=1, s=0.4):
    intensive_solution = odeint(intensive_dynamics, init_conditions[:2], t, args=(A, alpha, delta, n, s))
    # Your code here
    plt.figure(figsize=(6,4))
    plt.plot(t,intensive_solution[:,0], label='y')
    plt.plot(t,intensive_solution[:,1], label='k')
    plt.xlabel("Time (t)")
    plt.ylabel("Output / capital")
    plt.legend()
    plt.grid(True)
    plt.show()
@interact(A=(0,10,0.1), alpha=(0,1,0.01), delta=(0,1,0.01), n=(0,4,0.1), s=(0,1,0.01))
def plot_extensive_dynamics(A=2, alpha=0.5, delta=0.15, n=0.4, s=0.35):
    extensive_solution = odeint(extensive_dynamics, init_conditions, t, args=(A, alpha, delta, n, s))
    # Your code here
    plt.figure(figsize=(6,4))
    plt.plot(t,extensive_solution[:,0], label='Y')
    plt.plot(t,extensive_solution[:,1], label='K')
    plt.plot(t,extensive_solution[:,2], label='L')
    plt.xlabel("Time (t)")
    plt.ylabel("Output / capital / Labor")
    plt.legend()
    plt.grid(True)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='A', max=10.0), FloatSlider(value=0.4, description='a…

interactive(children=(FloatSlider(value=2.0, description='A', max=10.0), FloatSlider(value=0.5, description='a…

## Section 2: The Steady State and Golden Rule


### Task 1: Find steady state capital and consumption per worker.

The steady state is the long-run equilibrium, where capital per worker and output per worker are constant. In other words, the steady state is the point where the change in capital per worker and output per worker are zero. Find the steady state level of capital $k^*$.

$$k_* = (\frac{sA}{n+ \delta})^{\frac{1}{1-\alpha}}$$

In [30]:
from sympy import symbols, Eq, solve
# Define all the symbols

A, k, alpha, s, y, delta, n = symbols('A k alpha s y delta n')

eq = s * A*k**alpha - (n+delta)*k# Write the equation here
k_star = solve(eq, k)[0]  # save the solution to k_star for later use
k_star

(A*s/(delta + n))**(-1/(alpha - 1))

Wit the steady state level of capital, we can find the steady state level of output per worker, and hance consumption per worker.

What is the consumption per worker in the steady state?
Recall that $y = A \cdot k^\alpha$ and $c = (1-s) \cdot y$.


In [24]:
c = symbols('c')
y = A*k_star**alpha
c = (1-s)*y
c

A*(1 - s)*((A*s/(delta + n))**(-1/(alpha - 1)))**alpha

### Task 2: The Golden Rule level of savings.

**Question:** How is the Golden Rule savings level defined?\
**Answer:** $s_{gold} = \beta$ is the savings rate that leads to $k_{gold}$, the optimal amount of capital per capita to maximize consumption per capita


**Question:** If it the real savings rate is higher than the Golden Rule level, what happens to consumption per worker?\
**Answer:** With a savings rate higher than the Golden rule level, the amount of capital will be above equilibrium which leads to the depreciation gain being higher than the output gain and so the level of consumption we could have will instead be used to account for the larger depreciation rate


Derive $s_\text{gold}$.

Steps:
1. Write down the equation for consumption per worker, $c$ at the steady state $k = k^*$.
2. Take the derivative of $c$ with respect to $s$.
3. Set the derivative equal to zero and solve for $s$.


$$c_* = (1-s)\cdot f(k_*) = f(k_*) - sf(k_*) = f(k_*) - (n+\delta)k_* = Ak_*^{\alpha}-(n+\delta)k_*$$


$$= A \cdot ((\frac{sA}{n+\delta})^{\frac{1}{1-\alpha}})^{\alpha} - (n+\delta)\cdot ((\frac{sA}{n+\delta})^{\frac{1}{1-\alpha}})  = A \cdot (\frac{sA}{n+\delta})^{\frac{\alpha}{1-\alpha}} - (n+\delta)\cdot ((\frac{sA}{n+\delta})^{\frac{1}{1-\alpha}})$$

In [41]:
from sympy import diff

c_star = A * k_star**alpha - n*k_star - delta * k_star# hint use k_star saved above
dcds = diff(c_star, s)# your code here
# solve
s_gold = solve(dcds, s)[0]
# s_gold
c_star


A*((A*s/(delta + n))**(-1/(alpha - 1)))**alpha - delta/(A*s/(delta + n))**(1/(alpha - 1)) - n/(A*s/(delta + n))**(1/(alpha - 1))

## Section 3: Adding Human Capital

Human capital refers to the skills, knowledge, experience, and abilities possessed by workers. In the context of economic growth, human capital is essential because it enhances the productivity of labor.\
Traditional physical capital $K$ like machinery and buildings is not the sole driver of economic output; the skill level and efficiency of the workforce are equally important.


A production function incorporating human capital is:

$$
Y = A \cdot K^\alpha \cdot H^\beta \cdot L^{1-\alpha-\beta}
$$

The equations for capital and labor accumulation are:
$$
\frac{dK}{dt} = s_k \cdot Y - \delta_k \cdot K
$$
$$
\frac{dH}{dt} = s_h \cdot Y - \delta_h \cdot H
$$
$$
\frac{dL}{dt} = n \cdot L
$$

### Task 1: Explain the meaning of each variable




Let's explain the new ones that were introduced in this section:
<br>
We now have both $\alpha$ and $\beta$, I will assume that we have $\alpha + \beta \leq 1$ although I'm not entirely sure if this makes sense.
<br>
$\beta$ : The 'importance' of human capital compared to capital and labor force
<br>
$s_k$ : What we had as $s$ before this section. The savings rate for capital
<br>
$\delta_k$ : What we had as $\delta$ before this section. The depreciation rate for capital
<br>
$s_h$ : The savings rate for human capital. Can be interpreted as what is invested in the growth of human skill level and efficiency
<br>
$\delta_h$ : The depreciation rate for human capital. Can be interpreted as the rate at which human skills and competencies disappear with time


### Task 2: Write down the intensive form of the model.



$$\frac{dk}{dt} = (\frac{K}{L})´$$

$$= \frac{dK}{dt}L^{-1} + K \cdot (L^{-1})´$$

$$= \frac{s_k \cdot Y - \delta_k \cdot K}{L} - K \cdot L^{-2} \cdot nL$$

$$= \frac{s_k Y - \delta_k K - nK}{L}$$

$$\frac{dk}{dt}= s_k y - \delta_k k - nk = s_k y - (\delta_k + n)k$$

$$\frac{dh}{dt} = \frac{dH}{dt}L^{-1} + H \cdot (L^{-1})´ = \frac{s_h Y-\delta_h H}{L} - H \cdot L^{-2} \cdot nL$$

$$= s_h y - \delta_h h - n h = s_h y - (\delta_h + n) h$$

$$\frac{dl}{dt} = 0$$

$$y = \frac{Y}{L} = A\frac{K^{\alpha}\cdot H^{\beta} \cdot L^{1-\alpha-\beta}}{L^{\alpha}\cdot L^{\beta} \cdot L^{1-\alpha-\beta}}$$

$$y = Ak^{\alpha}h^{\beta}$$

In [44]:
def output_per_worker_human(A, k, h, alpha, beta):
    return A * k**alpha * h**beta

def intensive_dynamics_human(y,t,A, alpha, beta, delta_k, delta_h, n, s_k, s_h):
    output = output_per_worker_human(A, y[1],y[2], alpha, beta)
    dk_dt = s_k * y[0] - n* y[1] - delta_k * y[1]
    dh_dt = s_h*y[0] - n*y[2] - delta_h*y[2]
    return [output, dk_dt, dh_dt]

In [46]:
t = np.linspace(0, 5, 50)
init_conditions_human = [100, 70, 50]

@interact(A=(0,10,0.1), alpha=(0,1,0.01),beta=(0,1,0.01), delta_k=(0,1,0.01),delta_h=(0,1,0.01), n=(0,4,0.1), s_k=(0,1,0.01), s_h=(0,1,0.01))
def plot_wage_dynamics(A=1, alpha=0.4, beta = 0.2, delta_k=0.2,delta_h = 0.1, n=1, s_k=0.4, s_h=0.2):
    intensive_solution_human = odeint(intensive_dynamics_human, init_conditions_human, t, args=(A, alpha, beta, delta_k, delta_h, n, s_k, s_h))
    # Your code here
    plt.figure(figsize=(6,4))
    plt.plot(t,intensive_solution_human[:,0], label='y')
    plt.plot(t,intensive_solution_human[:,1], label='k')
    plt.plot(t,intensive_solution_human[:,2], label='h')
    plt.xlabel("Time (t)")
    plt.ylabel("Output / capital / human capital")
    plt.legend()
    plt.grid(True)
    plt.show()

interactive(children=(FloatSlider(value=1.0, description='A', max=10.0), FloatSlider(value=0.4, description='a…